In [7]:
import numpy as np

class MoveError(Exception):
    pass

class Game:
    
    MAX_MOVES = 2
    
    def __init__(self, start): # start = starting player (1 or -1)
        
        self.board = np.zeros((7, 7)); # initialize a 7x7 board populated with 0's
        
        self.board[2:5, 2:5] = 1 # populate the center 3x3 tiles with 1's (gold ships)
        self.board[3][3] = 2 # populate the center tile with 2 (flag)
        self.board[2:5, [0, 6]] = -1 # populate the middle 3 outer horizontal tiles with -1 (silver ships)
        self.board[[0, 6], 2:5] = -1 # populate the middle 3 outer vertical tiles with -1 (silver ships)
        
        self.goldPieces = [(i, j) for i in range(2, 5) for j in range(2, 5)] # contains all the positions of the gold pieces
        self.goldPieces.remove((3, 3)) # flag is a special piece
        self.silverPieces = [(i, j) for i in range(7) for j in range(7) if self.board[i, j] == -1] # contains all the positions of the silver pieces
        self.flag = (3,3) # flag is a special piece
        
        self.toMove = start # player to move first
        self.numMoves = 0 # number of moves already made by the player
        
        self.gameState = 0; # gamestate => 0 if game is in progress, 1 if gold wins, -1 if silver wins
    
    def move(self, originalPos, nextPos):
        old_x,old_y = originalPos;
        new_x, new_y = nextPos;
        
        if not (0 <= old_x <= 6 and 0 <= old_y <= 6 and 0 <= new_x <= 6 and 0 <= new_y <= 6) : # either the old pos or new pos is out of bounds for the 7x7 board
            raise MoveError("Out of Bounds")
        
        if (self.board[old_x][old_y] != self.toMove) or (self.board[old_x][old_y] + self.toMove == 3) : # piece selected isn't the current player's pieces
            #explanation, self.board[old_x][old_y] + self.toMove will only ever equal 3 if the selected piece is the flag, and the player to move is gold
            raise MoveError("Invalid Piece to Move")
        
        euclidian_distance = (old_x - new_x)**2 + (old_y - new_y)**2 # euclidian distance squared
        
        if euclidian_distance > 2 or euclidian_distance == 0 : # if tried to move in place, or tried to move too far away
            raise MoveError("Invalid Movement. Tried to move too far, or in place!")
        
        # finished testing general invalidations, now decide what move is being made
        
        if euclidian_distance == 1 : # horizontal move 
            if (self.board[new_x][new_y] != 0) : # must move into an empty space
                raise MoveError("Invalid Movement. Space is already occupied")

            self.board[new_x][new_y] = self.board[old_x][old_y]
            self.board[old_x][old_y] = 0
            
            if self.toMove == 1 : # gold pieces
                if self.board[old_x][old_y] == 1 :
                    self.goldPieces.remove((old_x,old_y))
                    self.goldPieces.append((new_x,new_y))
                elif (self.numMoves == 0) : # moving flag. since it's not a piece, if it passed the earlier checks, it must be the flag
                    self.flag = (new_x,new_y) # will only ever enter this condition if the player can select the flag piece, aka, the player owns the gold pieces
                    self.numMoves += 1
                else :
                    raise MoveError("Invalid Movement. Not enough movement left to move flag")
                
                if new_x == 0 or new_x == 6 or new_y == 0 or new_y == 6 :
                    self.gameState = 1 # gold piece reached the border, gold won
            else : # silver pieces
                self.silverPieces.remove((old_x,old_y))
                self.silverPieces.append((new_x,new_y))
            
            self.numMoves += 1
            
            return True # successful move
        
        # if euclidian distance isn't 1, because of the test done earlier to make sure the movement wasn't past what is allowed, it must be 2
        
        # diagonal capture
        
        if self.board[new_x][new_y] == 0 :
            raise MoveError("Invalid Movement. can't move into a diagonal empty space")
        if self.board[new_x][new_y] * self.board[old_x][old_y] > 0 : # will only result in positive if the pieces are of the same side
            raise MoveError("Invalid Movement. can't capture your own pieces")
        if self.numMoves != 0 :
            raise MoveError("Invalid Movement. Not enough movement to capture")
        
        if self.toMove == 1 and self.board[new_x][new_y] == -1 : # gold pieces capturing a silver piece
            if self.board[old_x][old_y] == 1 :
                self.goldPieces.remove((old_x,old_y))
                self.goldPieces.append((new_x,new_y))
            else : # moving flag
                self.flag = (new_x,new_y) # will only ever enter this condition if the player can select the flag piece, aka, the player owns the gold pieces
            
            self.silverPieces.remove((new_x,new_y)) # silver piece was captured
            
        elif self.toMove == -1 and self.board[new_x][new_y] == 1 :
            self.silverPieces.remove((old_x,old_y))
            self.silverPieces.append((new_x,new_y))
            
            self.goldPieces.remove((new_x,new_y)) # gold piece was captured
            
        elif self.toMove == -1 and self.board[new_x][new_y] == 2 :
            self.silverPieces.remove((old_x,old_y))
            self.silverPieces.append((new_x,new_y))
            
            self.flag = None;
            self.gameState = -1 #flag was captured, silver won
            
        self.numMoves += 2; # capturing takes all the move
    

In [9]:
game = Game(2)

game.goldPieces

[(2, 2), (2, 3), (2, 4), (3, 2), (3, 3), (3, 4), (4, 2), (4, 3), (4, 4)]

In [83]:
import tkinter as tk


class BattleScreen(tk.Frame):
    def __init__(self, root):
        tk.Frame.__init__(self, root)
        
        self.buttons = [[None] * 7 for _ in range(7)]

        for row in range(7):
            for col in range(7):
                button = tk.Button(self, width=4, height=3, command=lambda r=row, c=col: self.on_button_click(r, c))
                button.grid(row=row, column=col)
                self.buttons[row][col] = button

    def on_button_click(self, row, col):
        print(f"Button clicked at row {row}, column {col}")

In [84]:
class Controls(tk.Frame):
    def __init__(self, root, battle_screen):
        tk.Frame.__init__(self, root)

        self.battle_screen = battle_screen

        self.quit_button = tk.Button(self, text="Quit", width=6, command=root.destroy)
        self.quit_button.pack()


In [85]:
class GameApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Breakthru")

        self.battle_screen = BattleScreen(self.root)
        self.battle_screen.pack()

        self.controls = Controls(self.root, self.battle_screen)
        self.controls.pack()

In [86]:
def main():
    root = tk.Tk()
    app = GameApp(root)
    root.mainloop()

In [87]:
main()

Button clicked at row 2, column 5
Button clicked at row 3, column 3
Button clicked at row 5, column 4
